## Basic Structured Operations


### Step 1: Initialize PySpark Session


In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
# Create a Spark session
spark = SparkSession.builder.appName("day2").getOrCreate()


your 131072x1 screen size is bogus. expect trouble
23/08/31 08:31:19 WARN Utils: Your hostname, SUSHAN resolves to a loopback address: 127.0.1.1; using 172.31.35.44 instead (on interface eth0)
23/08/31 08:31:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 08:31:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2: Load the Dataset


In [3]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "occupation.csv"  # Replace with the actual path
occupation = spark.read.csv(data_path, header=True, inferSchema=True)


In [4]:
occupation.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



### Problem 1: Selecting Specific Columns
Problem: Select the "user_id," "age," and "occupation" columns from the occupation DataFrame.

In [5]:
occupation.select('user_id', 'age', 'occupation').show()

+-------+---+-------------+
|user_id|age|   occupation|
+-------+---+-------------+
|      1| 24|   technician|
|      2| 53|        other|
|      3| 23|       writer|
|      4| 24|   technician|
|      5| 33|        other|
|      6| 42|    executive|
|      7| 57|administrator|
|      8| 36|administrator|
|      9| 29|      student|
|     10| 53|       lawyer|
|     11| 39|        other|
|     12| 28|        other|
|     13| 47|     educator|
|     14| 45|    scientist|
|     15| 49|     educator|
|     16| 21|entertainment|
|     17| 30|   programmer|
|     18| 35|        other|
|     19| 40|    librarian|
|     20| 42|    homemaker|
+-------+---+-------------+
only showing top 20 rows



### Problem 2: Filtering Rows based on Condition
Problem: Find the users who are older than 30 years from the occupation DataFrame.

In [6]:
from pyspark.sql.functions import col
occupation.select("*").filter(col("age")>30).show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      2| 53|     F|        other|   94043|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemaker|   95660|
|     25| 39|     M|     engineer|   55107|
|     26| 49|     M|     engineer|   21044|
|     27| 40|     F|    librarian|   30030|
|     28| 32|     M|       writer|   55369|
|     29| 41|     M|   programmer|   94043|
|     34| 38|     F|administrator|   42141|
|     39| 41|     M|entertainmen

### Problem 3: Counting and Grouping
Problem: Count the number of users in each occupation from the occupation DataFrame.

In [7]:
from pyspark.sql.functions import count
occupation.groupBy("occupation").agg(count("user_id").alias("user_count")).show()

+-------------+----------+
|   occupation|user_count|
+-------------+----------+
|    librarian|        51|
|      retired|        14|
|       lawyer|        12|
|         none|         9|
|       writer|        45|
|   programmer|        66|
|    marketing|        26|
|        other|       105|
|    executive|        32|
|    scientist|        31|
|      student|       196|
|     salesman|        12|
|       artist|        28|
|   technician|        27|
|administrator|        79|
|     engineer|        67|
|   healthcare|        16|
|     educator|        95|
|entertainment|        18|
|    homemaker|         7|
+-------------+----------+
only showing top 20 rows



### Problem 4: Adding a New Column
Problem: Add a new column "age_group" to the occupation DataFrame based on the age of the users. Divide users into age groups: "18-25", "26-35", "36-50", and "51+".

In [8]:
from pyspark.sql.functions import when

conditions = [
    (col("age").between(18, 25), "18-25"),
    (col("age").between(26, 35), "26-35"),
    (col("age").between(36, 50), "36-50"),
    (col("age") >= 51, "51+")
]

occupation_with_age_group = occupation.withColumn("age_group", 
                                                  when(conditions[0][0], conditions[0][1])
                                                 .when(conditions[1][0], conditions[1][1])
                                                 .when(conditions[2][0], conditions[2][1])
                                                 .when(conditions[3][0], conditions[3][1])
                                                 .otherwise("Unknown"))

occupation_with_age_group.show()

+-------+---+------+-------------+--------+---------+
|user_id|age|gender|   occupation|zip_code|age_group|
+-------+---+------+-------------+--------+---------+
|      1| 24|     M|   technician|   85711|    18-25|
|      2| 53|     F|        other|   94043|      51+|
|      3| 23|     M|       writer|   32067|    18-25|
|      4| 24|     M|   technician|   43537|    18-25|
|      5| 33|     F|        other|   15213|    26-35|
|      6| 42|     M|    executive|   98101|    36-50|
|      7| 57|     M|administrator|   91344|      51+|
|      8| 36|     M|administrator|   05201|    36-50|
|      9| 29|     M|      student|   01002|    26-35|
|     10| 53|     M|       lawyer|   90703|      51+|
|     11| 39|     F|        other|   30329|    36-50|
|     12| 28|     F|        other|   06405|    26-35|
|     13| 47|     M|     educator|   29206|    36-50|
|     14| 45|     M|    scientist|   55106|    36-50|
|     15| 49|     F|     educator|   97301|    36-50|
|     16| 21|     M|entertai

### Problem 5: Creating DataFrames and Converting to Spark Types
Problem: Given the provided code snippet, create a DataFrame df using the given data and schema. The schema includes columns for firstname, middlename, lastname, id, gender, and salary. After creating the DataFrame, print its schema and display its content without truncation.

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("firstname", StringType(), nullable=True),
    StructField("middlename", StringType(), nullable=True),
    StructField("lastname", StringType(), nullable=True),
    StructField("id", StringType(), nullable=True),
    StructField("gender", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True)
])

data = [
    ("James", " ", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", " ", "40288", "M", 4000),
    ("Robert", " ", "Williams", "42114", "M", 4000),
    ("Maria", "Anne", "Jones", "39192", "F", 4000),
    ("Jen", "Mary", "Brown", " ", "F", -1)
]

df_n = spark.createDataFrame(data, schema=schema)

# Show the schema
df_n.printSchema()

# Show the content of the DataFrame without truncation
df_n.show(truncate=False)


root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



### Problem 6: Adding and Renaming Columns
Problem: Add a new column "gender" to the existing DataFrame and rename the "Age" column to "Years".

In [10]:
from pyspark.sql.functions import lit

df = occupation.withColumn("gender",lit("Unknown"))
df2 = df.withColumnRenamed("age","Years")
df2.show()

+-------+-----+-------+-------------+--------+
|user_id|Years| gender|   occupation|zip_code|
+-------+-----+-------+-------------+--------+
|      1|   24|Unknown|   technician|   85711|
|      2|   53|Unknown|        other|   94043|
|      3|   23|Unknown|       writer|   32067|
|      4|   24|Unknown|   technician|   43537|
|      5|   33|Unknown|        other|   15213|
|      6|   42|Unknown|    executive|   98101|
|      7|   57|Unknown|administrator|   91344|
|      8|   36|Unknown|administrator|   05201|
|      9|   29|Unknown|      student|   01002|
|     10|   53|Unknown|       lawyer|   90703|
|     11|   39|Unknown|        other|   30329|
|     12|   28|Unknown|        other|   06405|
|     13|   47|Unknown|     educator|   29206|
|     14|   45|Unknown|    scientist|   55106|
|     15|   49|Unknown|     educator|   97301|
|     16|   21|Unknown|entertainment|   10309|
|     17|   30|Unknown|   programmer|   06355|
|     18|   35|Unknown|        other|   37212|
|     19|   4

### Problem 7: Filtering Rows and Sorting
Problem: Filter out users who are younger than 30 years and sort the DataFrame by age in descending order.

In [11]:
df3 = df2.select("*").filter(col("Years") < 30).orderBy(col("Years").desc())
df3.show()

+-------+-----+-------+----------+--------+
|user_id|Years| gender|occupation|zip_code|
+-------+-----+-------+----------+--------+
|    222|   29|Unknown|programmer|   27502|
|     43|   29|Unknown| librarian|   20854|
|     45|   29|Unknown|programmer|   50233|
|    109|   29|Unknown|     other|   55423|
|    193|   29|Unknown|   student|   49931|
|      9|   29|Unknown|   student|   01002|
|    229|   29|Unknown| librarian|   22903|
|    250|   29|Unknown| executive|   95110|
|    297|   29|Unknown|  educator|   98103|
|    299|   29|Unknown|    doctor|   63108|
|    354|   29|Unknown| librarian|   48197|
|    365|   29|Unknown|    lawyer|   20009|
|    404|   29|Unknown|programmer|   55108|
|    407|   29|Unknown|  engineer|   03261|
|    467|   29|Unknown|  engineer|   91201|
|    473|   29|Unknown|   student|   94708|
|    478|   29|Unknown|     other|   10019|
|    483|   29|Unknown| scientist|   43212|
|    490|   29|Unknown|    artist|   V5A2B|
|    495|   29|Unknown|  enginee

### Problem 8: Repartitioning and Collecting Rows
Problem: Repartition the DataFrame into 2 partitions without shuffling the data, then collect and display all rows in the driver and print number of partitions

In [12]:
# df_repartitioned = df_n.repartition(2)
# rows = df_repartitioned.collect()

# Using repartition will shuffle the data. Since we need without shuffle, we need to use coalesce

df_coalesced = df_n.coalesce(2)
rows = df_coalesced.collect()
for row in rows:
    print(row)

Row(firstname='James', middlename=' ', lastname='Smith', id='36636', gender='M', salary=3000)
Row(firstname='Michael', middlename='Rose', lastname=' ', id='40288', gender='M', salary=4000)
Row(firstname='Robert', middlename=' ', lastname='Williams', id='42114', gender='M', salary=4000)
Row(firstname='Maria', middlename='Anne', lastname='Jones', id='39192', gender='F', salary=4000)
Row(firstname='Jen', middlename='Mary', lastname='Brown', id=' ', gender='F', salary=-1)


In [13]:
num_partitions = df_coalesced.rdd.getNumPartitions()
print("Number of partitions =", num_partitions)


Number of partitions = 2


### Additional questions:

Use both spark SQL and Pyspark to obtain answer wherever relevant

#### Filter out rows where the age is greater than 30 and create a new DataFrame. Then, add a new column named "is_elderly" with a value of "True" for these rows and "False" otherwise.Rename the "gender" column to "sex".

In [14]:
# Spark SQL
occupation.createOrReplaceTempView("occupation_table")

# Apply Spark SQL operations
result = spark.sql("""
    SELECT user_id, Age,
       Gender AS sex, 
       occupation, zip_code,
       CASE WHEN Age > 30 THEN True ELSE False END AS is_elderly
FROM occupation_table;
""")

# Show the result
result.show()

+-------+---+---+-------------+--------+----------+
|user_id|Age|sex|   occupation|zip_code|is_elderly|
+-------+---+---+-------------+--------+----------+
|      1| 24|  M|   technician|   85711|     false|
|      2| 53|  F|        other|   94043|      true|
|      3| 23|  M|       writer|   32067|     false|
|      4| 24|  M|   technician|   43537|     false|
|      5| 33|  F|        other|   15213|      true|
|      6| 42|  M|    executive|   98101|      true|
|      7| 57|  M|administrator|   91344|      true|
|      8| 36|  M|administrator|   05201|      true|
|      9| 29|  M|      student|   01002|     false|
|     10| 53|  M|       lawyer|   90703|      true|
|     11| 39|  F|        other|   30329|      true|
|     12| 28|  F|        other|   06405|     false|
|     13| 47|  M|     educator|   29206|      true|
|     14| 45|  M|    scientist|   55106|      true|
|     15| 49|  F|     educator|   97301|      true|
|     16| 21|  M|entertainment|   10309|     false|
|     17| 30

In [15]:
# Pyspark
result = occupation.withColumn("is_elderly", when(col("Age") > 30, True).otherwise(False)).withColumnRenamed("Gender", "sex")
result.show()


+-------+---+---+-------------+--------+----------+
|user_id|age|sex|   occupation|zip_code|is_elderly|
+-------+---+---+-------------+--------+----------+
|      1| 24|  M|   technician|   85711|     false|
|      2| 53|  F|        other|   94043|      true|
|      3| 23|  M|       writer|   32067|     false|
|      4| 24|  M|   technician|   43537|     false|
|      5| 33|  F|        other|   15213|      true|
|      6| 42|  M|    executive|   98101|      true|
|      7| 57|  M|administrator|   91344|      true|
|      8| 36|  M|administrator|   05201|      true|
|      9| 29|  M|      student|   01002|     false|
|     10| 53|  M|       lawyer|   90703|      true|
|     11| 39|  F|        other|   30329|      true|
|     12| 28|  F|        other|   06405|     false|
|     13| 47|  M|     educator|   29206|      true|
|     14| 45|  M|    scientist|   55106|      true|
|     15| 49|  F|     educator|   97301|      true|
|     16| 21|  M|entertainment|   10309|     false|
|     17| 30

#### Calculate the average age of male and female users separately. Present the result in a new DataFrame with columns "gender" and "avg_age".

In [17]:
# Spark SQL
occupation.createOrReplaceTempView("occupation_t")

ans = spark.sql("""
       SELECT gender,
       AVG(age) AS avg_age
       FROM occupation_t
       GROUP BY gender
""")

ans.show()

+------+------------------+
|gender|           avg_age|
+------+------------------+
|     F| 33.81318681318681|
|     M|34.149253731343286|
+------+------------------+



In [20]:
# Pyspark
from pyspark.sql.functions import avg

ans = occupation.groupBy("gender").agg(avg("age").alias("avg_age"))
ans.show()


+------+------------------+
|gender|           avg_age|
+------+------------------+
|     F| 33.81318681318681|
|     M|34.149253731343286|
+------+------------------+



#### Add a new column named "full_name" to the dataset by concatenating the "user_id" and "occupation" columns. Then, rename the "zip_code" column to "postal_code" in the same DataFrame.

In [22]:
# Spark SQL

ans = spark.sql("""
       SELECT *,
       CONCAT(user_id, occupation) AS full_name,
       zip_code AS postal_code
       FROM occupation_t;
""")

ans.show()

+-------+---+------+-------------+--------+---------------+-----------+
|user_id|age|gender|   occupation|zip_code|      full_name|postal_code|
+-------+---+------+-------------+--------+---------------+-----------+
|      1| 24|     M|   technician|   85711|    1technician|      85711|
|      2| 53|     F|        other|   94043|         2other|      94043|
|      3| 23|     M|       writer|   32067|        3writer|      32067|
|      4| 24|     M|   technician|   43537|    4technician|      43537|
|      5| 33|     F|        other|   15213|         5other|      15213|
|      6| 42|     M|    executive|   98101|     6executive|      98101|
|      7| 57|     M|administrator|   91344| 7administrator|      91344|
|      8| 36|     M|administrator|   05201| 8administrator|      05201|
|      9| 29|     M|      student|   01002|       9student|      01002|
|     10| 53|     M|       lawyer|   90703|       10lawyer|      90703|
|     11| 39|     F|        other|   30329|        11other|     

In [23]:
# Pyspark
from pyspark.sql.functions import concat

result = df.withColumn("full_name", concat(col("user_id"), col("occupation"))) \
           .withColumnRenamed("zip_code", "postal_code")
result.show()

+-------+---+-------+-------------+-----------+---------------+
|user_id|age| gender|   occupation|postal_code|      full_name|
+-------+---+-------+-------------+-----------+---------------+
|      1| 24|Unknown|   technician|      85711|    1technician|
|      2| 53|Unknown|        other|      94043|         2other|
|      3| 23|Unknown|       writer|      32067|        3writer|
|      4| 24|Unknown|   technician|      43537|    4technician|
|      5| 33|Unknown|        other|      15213|         5other|
|      6| 42|Unknown|    executive|      98101|     6executive|
|      7| 57|Unknown|administrator|      91344| 7administrator|
|      8| 36|Unknown|administrator|      05201| 8administrator|
|      9| 29|Unknown|      student|      01002|       9student|
|     10| 53|Unknown|       lawyer|      90703|       10lawyer|
|     11| 39|Unknown|        other|      30329|        11other|
|     12| 28|Unknown|        other|      06405|        12other|
|     13| 47|Unknown|     educator|     

#### Filter out rows where occupation is 'technician', select only the "user_id" and "age" columns, and then add a new column "age_diff" that calculates the difference between the user's age and the average age in the dataset.

In [24]:
# Spark SQL
ans = spark.sql("""
       SELECT user_id, age,
       age - AVG(age) OVER () AS age_diff
       FROM occupation_t
       WHERE occupation <> 'technician';
""")

ans.show()


23/08/31 08:19:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/31 08:19:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/31 08:19:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/31 08:19:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/31 08:19:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+---+-------------------+
|user_id|age|           age_diff|
+-------+---+-------------------+
|      2| 53| 18.921397379912662|
|      3| 23|-11.078602620087338|
|      5| 33| -1.078602620087338|
|      6| 42|  7.921397379912662|
|      7| 57| 22.921397379912662|
|      8| 36|  1.921397379912662|
|      9| 29| -5.078602620087338|
|     10| 53| 18.921397379912662|
|     11| 39|  4.921397379912662|
|     12| 28| -6.078602620087338|
|     13| 47| 12.921397379912662|
|     14| 45| 10.921397379912662|
|     15| 49| 14.921397379912662|
|     16| 21|-13.078602620087338|
|     17| 30| -4.078602620087338|
|     18| 35|  0.921397379912662|
|     19| 40|  5.921397379912662|
|     20| 42|  7.921397379912662|
|     21| 26| -8.078602620087338|
|     22| 25| -9.078602620087338|
+-------+---+-------------------+
only showing top 20 rows



In [ ]:
# Pyspark


#### Divide the dataset into two DataFrames: one with male users and another with female users. Repartition both DataFrames to have 2 partitions each. Then, union these two DataFrames together and display the resulting DataFrame.

#### Create and fill a new DataFrame named user_ratings with columns user_id and rating max 10 column. Both user_data and user_ratings share the user_id column. Combine these two DataFrames to create a new DataFrame that includes user information and their corresponding ratings. Make sure to keep only the users present in both DataFrames.